In [1]:
import pandas as pd
import nibabel as nib
import numpy as np
import os
from dice import dice

from utils import organizeFolders, organizeFiles, openImage

In [39]:
# Main Folder Dir
fp = 'C:\\Users\\colli\\Desktop\\lesionInterfaces\\Ground Truth Analysis'

# Pandas dataframe to store everything
"""
Shorthand meanings
    - 'vol' = original image volume
    - 'GT' = ground truth segmentation
    - 'MS' = Manual Segmentation
    - 'SC' = Smart Click Segmentation
"""
df = pd.DataFrame(columns=[
    # Patient ID
    'patient_id',

    # Voxel Count
    'vol_voxCnt',
    'GT_voxCnt',
    'MS_voxCnt',
    'SC_voxCnt',

    # Dice Score
    'GT-MS_dice',
    'GT-SC_dice',
    'MS_SC_dice',
])

# Maping file names to respective categories
file_map = {
    'vol' : 'dicom_vol',
    'GT' : 'GT_liver_cyst_vol',
    'MS' : 'DR_liver_cyst_vol',
    'SC' : 'DR_liver_cyst_vol_smart_click'}

inv_file_map = {v: k for k, v in file_map.items()}

In [40]:
# Get folder paths
folder_dir = organizeFolders(fp)

# Get file paths and info
for i in folder_dir:
    # Get File Paths
    file_dir = organizeFiles(i)
    patient_id = i.split('\\')[-1]
    print(patient_id)

    # Organize all numpy arrays into dictionary following the file_map
    np_files = {}

    for j in file_dir:
        np_files[ inv_file_map[j.split('\\')[-1].split('.nii')[0]]] = openImage(j)

    # Start geting information
    ## voxCnt

    temp = {}
    keyword = '_voxCnt'
    for k in np_files.keys():
        temp['patient_id'] = patient_id
        if k == 'vol':
            temp[k+keyword] = round(np_files[k].shape[0] * np_files[k].shape[1] * np_files[k].shape[2])
        else:
            temp[k+keyword] = round(np_files[k].sum())

    # Dice Score
    temp['GT-MS_dice'] = dice(np_files['GT'], np_files['MS']) # Ground Truth - Manual Segmentation
    temp['GT-SC_dice'] = dice(np_files['GT'], np_files['SC']) # Ground Truth - Smart Click Segmentation
    temp['MS_SC_dice'] = dice(np_files['MS'], np_files['SC']) # Manual Segmentation - Smart Click Segmentation


    df = df.append(pd.Series(temp, name=patient_id), ignore_index=True)

        

DEBUG: list directories - ['WC-IRB1308014251_003', 'WC-IRB1308014251_008', 'WC-IRB1308014251_011', 'WC-IRB1308014251_020', 'WC-IRB1308014251_025', 'WC-IRB1308014251_030']
WC-IRB1308014251_003
WC-IRB1308014251_008
WC-IRB1308014251_011
WC-IRB1308014251_020
WC-IRB1308014251_025
WC-IRB1308014251_030


In [41]:
df

,patient_id,vol_voxCnt,GT_voxCnt,MS_voxCnt,SC_voxCnt,GT-MS_dice,GT-SC_dice,MS_SC_dice
0,WC-IRB1308014251_003,6323200,5862,4810,4719,0.788043,0.888763,0.755378
1,WC-IRB1308014251_008,5529600,52839,51396,49033,0.985811,0.918820,0.921566
2,WC-IRB1308014251_011,4456448,74369,82829,71072,0.888154,0.976465,0.869390
3,WC-IRB1308014251_020,5529600,10851,10136,9929,0.901510,0.955438,0.891104
4,WC-IRB1308014251_025,2031616,13167,11486,11649,0.826999,0.937379,0.809077
5,WC-IRB1308014251_030,5529600,7565,4670,5158,0.587985,0.808772,0.545381


In [42]:
df.to_csv('dice_score.csv')